In [12]:
from pymatgen.core import Structure
#from pymatgen.core.structure import Structure
from pymatgen.core.periodic_table import get_el_sp
from maml.apps.bowsr.model.megnet import MEGNet
from maml.apps.bowsr.model.cgcnn import CGCNN
from maml.apps.bowsr.optimizer import BayesianOptimizer

structures = []

import os
        
structures.append(Structure.from_file("./MBT_mono_c2db.vasp"))

In [13]:
from aviary.cgcnn.utils import get_cgcnn_input
from aviary.wren.utils import count_wyks, get_aflow_label_spglib

In [14]:
import pandas as pd
def expected_radius(struct):
    element_list = struct.composition.chemical_system.split('-')
    element_list = [get_el_sp(e) for e in element_list]
    ele1, ele2 = sorted(element_list, key=lambda x: x.atomic_radius)[:2]
    return ele1.atomic_radius + ele2.atomic_radius

In [15]:
from maml.apps.bowsr.model.megnet import MEGNet
model = MEGNet() # or `model = CGCNN()`
#model = CGCNN()
import os, sys
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

def relaxStructure(structure):
    try:
        compressed_optimizer = BayesianOptimizer(
            model=model,
            structure=structure,
            relax_coords=True,
            relax_lattice=True,
            use_symmetry=False,
            seed = 128
        )

        compressed_optimizer.set_bounds()
        compressed_optimizer.optimize(n_init=80, n_iter=20, alpha=0.026 ** 2)
        radius = max(round(expected_radius(compressed_optimizer.structure) * 0.6, 2), 1.1)
        relaxed, _ = compressed_optimizer.get_optimized_structure_and_energy(radius=radius)
        print("Done")
        return relaxed
    except:
        print("ERROR")
        return -1

In [16]:
from tqdm import tqdm
tqdm_notebook().pandas()

NameError: name 'tqdm_notebook' is not defined

In [5]:
df = pd.DataFrame()
df['final_structure'] = structures
df['wyckoff'] = df['final_structure'].apply(get_aflow_label_spglib)
df['composition'] = df['final_structure'].apply(lambda x: x.composition.reduced_formula)
df.to_csv('MBTExample.csv', index = False, encoding='utf-8')

In [17]:
df['final_structure'] = df['final_structure'].apply(relaxStructure)

1/1 [==============================] - 0s 31ms/step
Done


In [18]:
#df1.to_csv('neuralNetworkInputMBTFamilyRELAXED.csv', index = False, encoding='utf-8')
df['wyckoff'] = df['final_structure'].apply(get_aflow_label_spglib)
df['composition'] = df['final_structure'].apply(lambda x: x.composition.reduced_formula)
df.to_csv('MBTExample.csv', index = False, encoding='utf-8')